In [1]:
from module_Plot import Plotting

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import ticker, cm, colors

import numpy as np
import pandas as pd
import os
import copy


plt.rcParams['axes.unicode_minus'] = False

In [2]:
loc_main = r'E:\data\_results\common_radar\used_data'

fn_20dBZ_Htop = os.path.join(loc_main, '_TG_2D_flag.csv')

base_legend = ['TRMM_PR', 'GPM_KuPR']

sfcType_sep = ['LandOcean_GPMKu']
sfcType = {0: 'Ocean', 1: 'Land'} # , 2: 'Coast'

rainType = {1: 'Stratiform', 2: 'Convective'}

self_colors = ['dodgerblue', 'salmon', 'limegreen', 'black']
linestyles = ['-', '--', ':', '-.']

fig_idxs = [f'({chr(ii)})' for ii in range(97, 123)]

properties = {'dBZ20_Htop': ['dBZ20_Htop (km)', 0.25, [0,20], 
                            ['dBZ20_Htop_TRMMKu', 'dBZ20_Htop_GPMKu', 'DLT_dBZ20_Htop', 'LandOcean_GPMKu'], 
                             1, [-10, 10]], 
             'dBZ35_Htop': ['dBZ35_Htop (km)', 0.25, [0,20], 
                            ['dBZ35_Htop_TRMMKu', 'dBZ35_Htop_GPMKu', 'DLT_dBZ35_Htop', 'LandOcean_GPMKu'], 
                            1, [-10, 10]], 
             'nearSurfdBZ': ['nearSurfdBZ (dBZ)', 1, [10,65], 
                             ['nearSurfdBZ_TRMMKu', 'nearSurfdBZ_GPMKu', 'DLT_nearSurfdBZ', 'LandOcean_GPMKu'], 
                             2, [-35, 35]],
             'nearSurfRain': [r'nearSurfRain (10$^x$ mm h$^{-1}$)', 0.1, [-1,3], 
                             ['nearSurfRain_TRMMKu', 'nearSurfRain_GPMKu', 'DLT_nearSurfRain', 'LandOcean_GPMKu'], 
                              0.1, [-3, 3]]}
thh_dBZ_GPM, thh_dBZ_TRMM = 18, 18  # dBZ

In [3]:
df_dBZ20 = pd.read_csv(fn_20dBZ_Htop, compression='gzip')
df_dBZ20.columns, df_dBZ20.shape

(Index(['Time_TRMMKu', 'Time_GPMKu', 'dlt_Dm', 'dlt_Tsec', 'orbitID_TRMMKu',
        'loc_x_TRMMKu', 'loc_y_TRMMKu', 'orbitID_GPMKu', 'loc_x_GPMKu',
        'loc_y_GPMKu', 'Lon_TRMMKu', 'Lat_TRMMKu', 'LocalHour_TRMMKu',
        'nearSurfRain_TRMMKu', 'dBZ20_Htop_TRMMKu', 'dBZ35_Htop_TRMMKu',
        'rainType_TRMMKu', 'HBB_TRMMKu', 'Lon_GPMKu', 'Lat_GPMKu',
        'LocalHour_GPMKu', 'nearSurfRain_GPMKu', 'dBZ20_Htop_GPMKu',
        'dBZ35_Htop_GPMKu', 'LandOcean_GPMKu', 'rainType_GPMKu', 'HBB_GPMKu',
        'nearSurfdBZ_TRMMKu', 'surfRainPhase_TRMMKu', 'nearSurfdBZ_GPMKu',
        'surfRainPhase_GPMKu', 'dBZ20_Npxl_TRMMKu', 'dBZ35_Npxl_TRMMKu',
        'dBZ20_Npxl_GPMKu', 'dBZ35_Npxl_GPMKu', 'profile_TRMM', 'profile_GPM',
        'GmT_Yloc_M', 'GmT_Yloc_L', 'GmT_Yloc_R', 'nearSurfRain_TRMMKu_log10',
        'nearSurfRain_GPMKu_log10'],
       dtype='object'),
 (217618, 42))

In [6]:
queries = {'row0':'(dlt_Dm>=0)', 'row1': '(rainType_TRMMKu==2)&(rainType_GPMKu==2)', 'row2': '(rainType_TRMMKu==1)&(rainType_GPMKu==1)',
           'col0':'(dlt_Dm>=0)', 'col1': '(LandOcean_GPMKu==1)', 'col2': '(LandOcean_GPMKu==0)'}

# constant defination of 1to1 comparison
bins = np.arange(0, 20.01, 0.25)

levs = np.array([1, 5 ]+list(np.arange(10,101, 5)))  # percentage
cb_labs = [1] + list(np.arange(10,101, 10))

tick_loc = [4,1,4,1]
var_LOC = ['ALL', 'Land', 'Ocean'] #, 'Coast'
var_CS = ['ALL', 'Convective', 'Stratiform']

from sklearn.metrics import mean_squared_error
####
def plot_9in1(fn, df_base, Vars, Varname='Topht20s'):
    df = df_base.query(f'({Vars[0]}>0) & ({Vars[1]}>0)')
    
    plot = Plotting(global_xy=False, fontsize=22)
    cmap = plot.icmap.set_cmap(rng=list(np.arange(6)+1), c_firstwhite=False, kind='pr1')  # copy.copy((cm.get_cmap('rainbow')))  
    
    nrows, ncols = 3, 3
    sns.set_context('talk', font_scale=1.4)
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(21, 21), sharex=True, sharey=True) #
    fig.subplots_adjust(wspace=0.15, hspace=0.15)
    
    ##     
    grid_dict = {}
    siglev = {}
    for idx in np.arange(nrows):        
        for jdx in np.arange(ncols):
            iax = ax[idx][jdx]
            iquery = '&'.join([queries[f'row{idx:.0f}'], queries[f'col{jdx:.0f}']])
#            print(idx, jdx, iquery)
            data = df.query(iquery)
            grid, Xedge, Yedge, _ = iax.hist2d(data[Vars[0]], data[Vars[1]], 
                                               bins=bins, visible=False)
            
            grid_dict[Vars[0].split('_T')[0]+'_'+var_CS[idx]+'_'+var_LOC[jdx]] = grid
            totDA, maxDA = data.shape[0], int(grid.max())
            pctDA = 100*grid/maxDA
            
            obj = iax.pcolormesh(Xedge, Yedge, pctDA,
                          cmap=cmap, shading='auto', 
                          norm=colors.BoundaryNorm(boundaries=levs, ncolors=cmap.N), 
                          edgecolors='none', alpha=1, zorder=2)
            obj.cmap.set_under('white')
            obj.cmap.set_over('firebrick')
            
            iax.axline(xy1=(bins.min(), bins.min()), xy2=(bins.max(), bins.max()),
                               linestyle='--', color='k')
            ###
            pcc = data[Vars].corr().to_numpy()[0, 1]
            rmse = np.sqrt(mean_squared_error(data[Vars[0]], data[Vars[1]]))
            bias = (data[Vars[1]]-data[Vars[0]]).mean()
#            print(f'{fig_idxs[jdx*ncols+idx]}: pcc= {pcc}, rmse= {rmse}, bias= {bias}')
            ###
            
            iax.text(x=0.01, y=0.98, s=f'{fig_idxs[idx*ncols+jdx]}', 
                 ma='left', ha='left', va='top', color='black', transform=iax.transAxes)
            iax.text(x=0.02, y=0.9, 
#                     s=f'PCC={pcc:.03f}\nTotal={totDA}\nNmax={maxDA}', 
                     s=f'PCC = {pcc:4.02f}\nBIAS = {bias:4.02f}\nRMSE = {rmse:4.02f}', fontsize=22, 
                     ma='left', ha='left', va='top', color='black', transform=iax.transAxes) #, fontsize=fz
            str_totDA = f'{totDA}' if totDA<1000 else f'{totDA//1000:.0f},'+f'{totDA%1000:.0f}'
            iax.text(x=0.98, y=0.15, 
                     s=f'Nmax = {maxDA}\nTotal = {str_totDA}', fontsize=22, 
                     ma='right', ha='right', va='top', color='black', transform=iax.transAxes) #, fontsize=fz
            
            iax.xaxis.set_major_locator(ticker.MultipleLocator(tick_loc[0]))
            iax.xaxis.set_minor_locator(ticker.MultipleLocator(tick_loc[1]))
            iax.yaxis.set_major_locator(ticker.MultipleLocator(tick_loc[2]))
            iax.yaxis.set_minor_locator(ticker.MultipleLocator(tick_loc[3]))
    
            if idx == 0:
                iax.set_title(var_LOC[jdx])
                
            if (idx==1) & (jdx == 0):
                iax.set_ylabel(f'{Varname} from {base_legend[0]} (km)')
            if (idx==2) & (jdx == 1):
                iax.set_xlabel(f'{Varname} from {base_legend[1]} (km)')
            if jdx == 2:
                iax.text(x=1.02, y=0.5, 
                     s=f'{var_CS[idx]}', rotation=90, 
                     ma='left', ha='left', va='center', color='black', transform=iax.transAxes)
            
            if (idx==2) & (jdx == 0):
                cb_ax = iax.inset_axes(bounds=[0.8, -0.3, 1.7, 0.07])
                cb = fig.colorbar(obj, cb_ax, orientation='horizontal') #shrink=cb_shrink, aspect=15, pad=0,  

                cb.set_ticks(cb_labs)
                cb.set_ticklabels(cb_labs)
                cb.ax.tick_params(direction='in', length=4)
                cb.ax.tick_params(which='minor', direction='in', length=2)
        #            tick_labs = plot.__shrink_tail_zeros__(cb.get_ticks())
        #            cb.ax.set_yticklabels(tick_labs, fontdict={'fontsize': plot.fontsize})  
                cb.set_label(' Percentage (%)', labelpad=1) #, fontdict={'fontsize': plot.fontsize}
            
    plot._save_fig(fn, fig)  
    del plot
    return grid_dict, siglev

In [7]:
df_dBZ20 = df_dBZ20.query(f'(dBZ20_Htop_TRMMKu > 0) & (dBZ20_Htop_GPMKu > 0) & (nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM}) & (nearSurfdBZ_GPMKu>={thh_dBZ_GPM})')
print(df_dBZ20.shape)
fn = 'coincident_1to1cmp_LO_CS_dBZ20_Htop'
grids, siglevs = plot_9in1(fn, df_dBZ20, ['dBZ20_Htop_TRMMKu', 'dBZ20_Htop_GPMKu'], Varname='Topht20s')

(97619, 42)


In [6]:
df_dBZ35 = df_dBZ20.query(f'(dBZ35_Htop_TRMMKu > 0) & (dBZ35_Htop_GPMKu > 0) & (nearSurfdBZ_TRMMKu>={thh_dBZ_TRMM}) & (nearSurfdBZ_GPMKu>={thh_dBZ_GPM})')
print(df_dBZ35.shape)
fn = 'coincident_1to1cmp_LO_CS_dBZ35_Htop'
grids, siglevs = plot_9in1(fn, df_dBZ35, ['dBZ35_Htop_TRMMKu', 'dBZ35_Htop_GPMKu'], Varname='Topht35s')

(21873, 42)


In [9]:
df_dBZ20 = df_dBZ20.query('dlt_Tsec <= 300')  # constrain in 10 minutes
df_dBZ20.shape

(52340, 42)